In [5]:
import numpy as np
import pandas as pd
import spacy

In [6]:
import fr_core_news_md

nlp = fr_core_news_md.load()

In [ ]:
def seg_doc(text):
    #V2 : regrouper les enfants isolés de la racine avec la racine

    doc = nlp(text)                      #pour traiter un ensemble de documents, utiliser docs =nlp.pipe(DOCS)

    
    segments = []
    roots = [token for token in doc if token.head == token]
    for root in roots: 
        seg_root = []
        for token in root.lefts:                  #segmenter en sous-arbre
            if len(list(token.subtree)) == 1:
                seg_root.append(token)
            else: 
                temp = []
                for descendant in token.subtree:            
                    if descendant.text == ",":        #segmenter en virgule
                        temp.append(descendant)
                        segments.append(temp)
                        temp = []
                    else:
                        temp.append(descendant)
                if len(temp) != 0:
                    segments.append(temp)
        position_root = len(segments)            
        seg_root.append(root)
        
        for token in root.rights:
            if len(list(token.subtree)) == 1:
                seg_root.append(token)
            else: 
                temp = []
                for descendant in token.subtree:            
                    if descendant.text == ",":        
                        temp.append(descendant)
                        segments.append(temp)
                        temp = []
                    else:
                        temp.append(descendant)
                if len(temp) != 0:
                    segments.append(temp)
        segments.insert(position_root, seg_root)            

    return segments

In [21]:
def seg_doc(text):
    #V2 : regrouper les enfants isolés de la racine avec la racine

    doc = nlp(text)                      #pour traiter un ensemble de documents, utiliser docs =nlp.pipe(DOCS)

    
    segments = []
    roots = [token for token in doc if token.head == token]
    for root in roots: 
        seg_root = []
        for token in root.lefts:                  #segmenter en sous-arbre
            if (len(list(token.subtree)) == 1) & (token.pos_ not in ['SPACE','PUNCT']):
                seg_root.append(token)
            else: 
                temp = []
                for descendant in token.subtree:            
                    if (descendant.text == ",") & (len(temp) != 0):        #segmenter en virgule
                        segments.append(temp)
                        temp = []
                    else:
                        if descendant.pos_ not in ['SPACE','PUNCT']:
                            temp.append(descendant)
                if len(temp) != 0:
                    segments.append(temp)
        position_root = len(segments)            
        seg_root.append(root)
        
        for token in root.rights:
            if (len(list(token.subtree)) == 1) & (token.pos_ not in ['SPACE','PUNCT']):
                seg_root.append(token)
            else: 
                temp = []
                for descendant in token.subtree:            
                    if (descendant.text == ",") & (len(temp) != 0):        
                        segments.append(temp)
                        temp = []
                    else:
                        if descendant.pos_ not in ['SPACE','PUNCT']:
                            temp.append(descendant)
                if len(temp) != 0:
                    segments.append(temp)
        segments.insert(position_root, seg_root)            

    return segments

def has_verb(doc):
    # déterminer si un segment contient un verbe
    has_verb = False
    for token in doc:
        if token.pos_ in ["VERB", "AUX"]:
            has_verb = True
            break
    return has_verb

def seg2sent(seg):
    # convertir un segment de token en un string de phrase
    phrase = ""
    for token in seg:
        phrase += token.text + " " 
    return phrase

def seg_doc_2(segments):
    #version flatten
    
    temp = [seg for seg in segments]
    for i,seg in enumerate(segments): 
        if has_verb(seg):                       # resegmenter les segments verbaux
            new_segs = seg_doc(seg2sent(seg))
            temp[i] = new_segs
        else:
            temp[i] = [segments[i]]
    return [seg for segs in temp for seg in segs]

def seg_auto(TEXT):
    #resegmenter les segments verbaux tant que le nombre de segments augmente
    segments = seg_doc(TEXT)
    c = 0
    while len(seg_doc_2(segments)) > len (segments):   
        segments = seg_doc_2(segments)
        c += 1
        print(f"segmenter {c} fois")
    return segments

In [18]:
df = pd.read_csv("/Users/rubing/Documents/Donnees/covax_content_sample_2000_medium_sized.csv").drop("Unnamed: 0", axis=1)

In [64]:
TEXT=df.extracted_text[3]
print(TEXT)

"Nous sommes dans une situation très préoccupante". Lors d'une visioconférence organisée ce mercredi après-midi, le président du Conseil scientifique n'a pas caché son inquiétude face à l'évolution des indicateurs de l'épidémie de Covid-19 à l'échelle nationale. "La reprise (de l'épidémie, NDLR) est plus importante en cette rentrée qu'on aurait pu l'imaginer" , reconnait Jean-François Delfraissy.
Même si ce rebond a encore "peu de retentissement sur le nombre d'hospitalisation ou d'admissions en réa", le risque, selon lui, serait d'être "faussement rassuré par un certain nombre de chiffres", alors même que "la progression est exponentielle" et que "l' on sait qu'il va y avoir une accélération très rapide dans un deuxième temps". 
"L'image que l'on a de pouvoir faire une séparation entre les anciens et les plus jeunes (tranche d'âge majoritairement concernée par les nouvelles contaminations, NDLR) n'aura qu'un temps. Il y a forcément des liens qui vont se produire".
Jean-François Delfra

In [65]:
segs = seg_auto(TEXT)

segmenter 1 fois
segmenter 2 fois
segmenter 3 fois
segmenter 4 fois
segmenter 5 fois
segmenter 6 fois
segmenter 7 fois


In [66]:
segs

[[Nous, sommes],
 [dans, une, situation, très, préoccupante],
 [Lors, d],
 [une, visioconférence],
 [organisée],
 [ce, mercredi, après-midi],
 [le, président, du, Conseil, scientifique],
 [n, a, pas, caché],
 [son, inquiétude],
 [face, à, l', évolution, des, indicateurs, de, l', épidémie],
 [de, Covid-19, à, l', échelle, nationale],
 [La, reprise, de, l', épidémie],
 [NDLR],
 [est, plus, importante],
 [en, cette, rentrée],
 [qu, on, aurait, pu],
 [l, imaginer],
 [reconnait],
 [Jean-François, Delfraissy],
 [Même, si],
 [ce, rebond],
 [a, encore, peu],
 [de, retentissement],
 [sur, le, nombre, d, hospitalisation, ou, d, admissions, en, réa],
 [le],
 [risque],
 [selon, lui],
 [serait, d],
 [être, faussement, rassuré],
 [par, un, certain, nombre, de, chiffres],
 [alors, même],
 [la, progression],
 [que, est, exponentielle],
 [l, on],
 [et, que, sait],
 [qu, il, va],
 [y, avoir],
 [une, accélération, très, rapide],
 [dans, un, deuxième, temps],
 [L, image, que, l],
 [on, a],
 [de, pouvoir],